In [146]:
# 加入必要的模块，没有的话先安装
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import pyperclip, re

In [89]:
# 配置浏览器，后台静默运行
option = webdriver.ChromeOptions()
# I use the following options as my machine is a window subsystem linux. 
# I recommend to use the headless option at least, out of the 3
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-sh-usage')
# option.add_argument('--start-maximized')
# Replace YOUR-PATH-TO-CHROMEDRIVER with your chromedriver location
driver = webdriver.Chrome('D://Program Files//Python//Python37//Scripts//chromedriver.exe', options=option)

<ipython-input-89-2fa1a36b7727>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D://Program Files//Python//Python37//Scripts//chromedriver.exe', options=option)


In [90]:
# 登陆账号
driver.get("https://www.letpub.com.cn/index.php?page=login")
email = driver.find_element_by_id("email")
email.clear()
email.send_keys("2628201733@qq.com")

password = driver.find_element_by_name("password")
password.clear()
password.send_keys("liuwei")
driver.execute_script("login()")

<ipython-input-90-0523c73fa561>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email = driver.find_element_by_id("email")
<ipython-input-90-0523c73fa561>:6: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  password = driver.find_element_by_name("password")


In [91]:
# 访问给定杂志的结果
url = pyperclip.paste()
page = driver.get(url) # Getting page HTML through request
# WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table_yjfx")))  # We are waiting for 5 seconds for our element with the attribute data-testid set as `firstListCardGroup-editorial`
driver.implicitly_wait(30) # 等待页面加载完毕
driver.refresh() # 刷新去除弹窗广告 

In [101]:
soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup

In [ ]:
info = soup.find_all("table", {"class": "table_yjfx"})[1]

In [191]:
rows = info.find('tbody').findAll("tr", recursive=False)

In [ ]:
rows

In [216]:
# 提取影响因子
number_re = re.compile(r'[0-9.]+')
row = info.find('tbody').find(lambda tag:tag.name == "tr" and "最新IF" in tag.text)
impact_factor = number_re.search(row.findAll("td")[1].text).group()
impact_factor



'7.561'

In [218]:
# 出版周期
row = info.find('tbody').find(lambda tag:tag.name == "tr" and "出版周期" in tag.text)
frequency = row.findAll("td")[1].text
frequency

'English'

In [219]:
# 年文章数
row = info.find('tbody').find(lambda tag:tag.name == "tr" and "年文章数" in tag.text)
NO_articles_year = number_re.search(row.findAll("td")[1].text).group()
NO_articles_year

'3298'

In [220]:
# SCI分区
def findDisplayedElementContent(eles):
    for ele in eles:
        try:
            ele['style']
            if 'display:none' not in ele['style']:
                result = ele.text
        except:
            print("Element do not have 'style' attribute")
    return result

row = info.find('tbody').find(lambda tag:tag.name == "tr" and "期刊SCI分区" in tag.text)
SCI_info = row.findAll("td")[1].findAll("tr")[1].find("td")
domain = SCI_info.findAll(text = True, recursive=False)[0]
partition = findDisplayedElementContent(SCI_info.findAll("span"))
SCI_partition = domain + partition
SCI_partition

'IMMUNOLOGY Q1'

In [221]:
# 中科院分区（基础版）
row = info.find('tbody').find(lambda tag:tag.name == "tr" and "最新基础版" in tag.text)
CST_info = row.findAll("td")[1].find("td")
CST_domain = CST_info.findAll(text = True, recursive=False)[0]
CST_part = findDisplayedElementContent(CST_info.findAll("span"))
CST_partition = CST_domain + CST_part
CST_partition

'医学 2区'

In [208]:
# 中科院分区（升级版）
row = info.find('tbody').find(lambda tag:tag.name == "tr" and "最新升级版" in tag.text)
CST_info_up = row.findAll("td")[1].find("td")
CST_domain_up = CST_info_up.findAll(text = True, recursive=False)[0]
CST_part_up = findDisplayedElementContent(CST_info_up.findAll("span"))
CST_partition_up = CST_domain_up + CST_part_up
CST_partition_up

'医学 2区'

In [213]:
# 将结果粘到剪贴板，以\t分隔便于贴到excel中
result = '\t'.join([impact_factor, frequency, NO_articles_year, SCI_partition, CST_partition, CST_partition_up])
pyperclip.copy(result)
print('IF: ' + str(impact_factor) + ', 出版周期: ' + str(frequency) + ', 年文章数：' + str(NO_articles_year) +', SCI分区:' + SCI_partition + ', 中科院分区(基础版):' + CST_partition + ', 中科院分区(升级版)' + CST_partition_up)

IF: 7.561, 出版周期: English, 年文章数：3298, SCI分区:IMMUNOLOGY Q1, 中科院分区(基础版):医学 2区, 中科院分区(升级版)医学 2区
